In [1]:
from os.path import join, dirname, exists
from os import makedirs
from itertools import chain, islice

from thesis_v2 import dir_dict
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200819_tang_generator,
)
from thesis_v2.analysis.io import collect_rcnn_k_bl_hal_analysis

In [2]:
import pandas as pd

In [3]:
f_main_result = join(dir_dict['analyses'], 'tang_refactored', '20200819', 'hal_tuning_complete.pkl')
if not exists(f_main_result):
    makedirs(dirname(f_main_result), exist_ok=True)
    df_main_result = collect_rcnn_k_bl_hal_analysis(
        fixed_keys = {
        'kernel_size_l23': 3,
        'kernel_size_l1': 9,
        'ff_1st_block': True,
        'pooling_ksize': 3,
        'pooling_type': 'avg',
        'bn_after_fc': False,
        'rcnn_bl_psize': 1,
        'rcnn_bl_ptype': None,
        'input_size': 63,
        'split_seed': 'legacy',
        'dataset_prefix': 'tang',
        'model_prefix': 'maskcnn_polished_with_rcnn_k_bl',
        'scale_name': '0.01',
        'scale': '0.01',
        'smoothness_name': '0.000005',
        'smoothness': '0.000005',
    },
        generator=chain(
            explored_models_20200819_tang_generator(with_source=True),
        ),
        total_num_param=26,
        train_size_mapping=dict(),
    )
    df_main_result.to_pickle(f_main_result)
    del df_main_result
df_main_result = pd.read_pickle(f_main_result)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   28.4s
/opt/conda/envs/leelab/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 834 tasks      | elapsed: 24.0min
[Parallel(n_jobs=-1)]: Done 1104 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done 1410 tasks      | elapsed: 40.6min
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed: 50.6min
[Parallel(n_jobs=-1)]: Done 2130 tasks      | elapsed: 61.3min
[Parallel(n

In [4]:
df_main_result

hal_tuning_analysis  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                       
relu     False                mse       0          2         8           1           cm-avg       350                       None   
                                                                                                  700                       None   
                                                                                                  1400                      None   
                                                                                     cm-last      350                       None   
                                                                                                  700                       None   
...                                                                                                                          ...   
softplus True                 poisson   1          6         16          1           legacy       700                       None   
                                                                                                  1400                      None   
                                                             32          1           legacy       350                       None   
                                                                                                  700                       None   
                                                                                                  1400                      None   

                                                                                                             hal_tuning_analysis_improved  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                                
relu     False                mse       0          2         8           1           cm-avg       350                                None   
                                                                                                  700                                None   
                                                                                                  1400                               None   
                                                                                     cm-last      350                                None   
                                                                                                  700                                None   
...                                                                                                                                   ...   
softplus True                 poisson   1          6         16          1           legacy       700                                None   
                                                                                                  1400                               None   
                                                             32          1           legacy       350                                None   
                                                                                                  700                                None   
                                                                                                  1400                               None   

                                                                                                             hal_tuning_analysis_improved_baseline  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                                         
relu     False                mse       0          2         8           1           cm-avg       350                                         None   
                                                                                                  700       

In [5]:
# verify against previous results using no parallel.
f_main_result_prev = join(dir_dict['analyses'], 'tang_refactored', '20200819', 'hal_tuning.pkl')
df_main_result_prev = pd.read_pickle(f_main_result_prev)

In [6]:
import numpy as np

In [7]:
def check_two_same_inner(x,y):
    if x is None and y is None:
        return True
    
    assert x is not None and y is not None
    assert x.keys() == y.keys()
    for k in x:
        assert np.array_equal(x[k], y[k])
    
    return True

def check_two_same(s1, s2):
    assert s1.index.equals(s2.index)
    ct = 0
    for (x, y) in zip(s1, s2):
        assert check_two_same_inner(x, y)
        ct += 1
    assert ct == 4752

In [10]:
assert len(df_main_result_prev.columns) == 5
assert len(df_main_result.columns) == 5
check_two_same(df_main_result_prev['hal_tuning_analysis'], df_main_result['hal_tuning_analysis'])
check_two_same(df_main_result_prev['hal_tuning_analysis_inverted'], df_main_result['hal_tuning_analysis_inverted'])
check_two_same(df_main_result_prev['hal_tuning_analysis_improved'], df_main_result['hal_tuning_analysis_improved'])
check_two_same(df_main_result_prev['hal_tuning_analysis_improved_baseline'], df_main_result['hal_tuning_analysis_improved_baseline'])